In [ ]:
!pip install BisPy
!pip install ogb
!pip install torch-geometric

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import random
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import pandas as pd
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB, Actor
import os
import bispy
from bispy import compute_maximum_bisimulation, Algorithms
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti_partition
from bispy.dovier_piazza_policriti.ranked_partition import RankedPartition
from bispy.dovier_piazza_policriti.dovier_piazza_policriti import dovier_piazza_policriti, collapse, build_block_counterimage, split_upper_ranks
from bispy.utilities.rank_computation import compute_rank
from bispy.utilities.graph_decorator import decorate_nx_graph, decorate_bispy_graph,as_bispy_graph,to_tuple_list
from bispy.utilities import *
import numpy as np
from sklearn.cluster import KMeans

from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
import matplotlib.pyplot as plt
import numpy as np

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from ogb.nodeproppred import PygNodePropPredDataset

dataset = PygNodePropPredDataset(name = "ogbn-arxiv")
# dataset = PygNodePropPredDataset(name = "ogbn-products")

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]
data = dataset[0] # pyg graph object
data.y = torch.squeeze(data.y)
# data.node_year = torch.squeeze(data.node_year)

# model_save_path_GCN = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_ogbn_arxiv_GCN.pt')
# model_save_path_GAT = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_ogbn_arxiv_GAT.pt')
# model_save_path_GraphSage = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_ogbn_arxiv_GraphSage.pt')

model_save_path_GCN = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_arxiv_GCN.pt')
model_save_path_GAT = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_arxiv_GAT.pt')
model_save_path_GraphSage = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_arxiv_GraphSage.pt')

data

Data(num_nodes=169343, edge_index=[2, 1166243], x=[169343, 128], node_year=[169343, 1], y=[169343])

In [ ]:
# WebKB
from torch_geometric.datasets import Planetoid, CoraFull, Yelp, WebKB

# dataset = WebKB(root='/tmp/WebKB', name='Wisconsin')
# dataset = Yelp(root='Yelp')

dataset = Yelp(root='/tmp/Yelp')
data = dataset[0]
data.y = torch.squeeze(data.y)
data

model_save_path_GCN = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_yelp_GCN.pt')
model_save_path_GAT = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_yelp_GAT.pt')
model_save_path_GraphSage = os.path.join('gdrive/My Drive/GNN_Sparsification/Pre_Trained_10%_yelp_GraphSage.pt')

In [ ]:
# Load tensor from .pt file
loaded_tensor = torch.load('ARXIV_SPAR.pt')
data.edge_index = loaded_tensor.edge_index

In [ ]:
data

Data(num_nodes=169343, edge_index=[2, 823148], x=[169343, 128], node_year=[169343, 1], y=[169343])

In [ ]:
import copy
from torch_geometric.utils import subgraph
train_mask = torch.rand(data.num_nodes) < 0.95
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
file_path = "gdrive/My Drive/GNN_Sparsification/5%_yelp_train_mask.pt"


# Save the binary tensor mask to a .pt file
torch.save(train_mask, file_path)

In [ ]:
# train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/10%_yelp_train_mask.pt")
train_mask = torch.load("gdrive/My Drive/GNN_Sparsification/5%_yelp_train_mask.pt")
test_mask = ~train_mask
tested_elements =  torch.nonzero(test_mask==True).squeeze()
tested_elements = tested_elements.tolist()

In [ ]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx


# Convert PyG graph to NetworkX graph
def pyg_to_networkx(pyg_data):
    edge_index = pyg_data.edge_index
    num_nodes = pyg_data.num_nodes
    edge_list = edge_index.t().tolist()
    G = nx.DiGraph()
    G.add_nodes_from(range(num_nodes))
    G.add_edges_from(edge_list)
    return G

networkx_graph = pyg_to_networkx(data)

# Diameter-based K-hops Induced Subgraph

In [ ]:
nodes_to_find_neighbors = tested_elements
neighbors = set()
up_to_k_hop_neighbors = []
up_to_k_hop_neighbors_list = []
k = 1
# G = nx.Graph()
G = nx.DiGraph()

for i in range(k):
  # Iterate through the nodes in your set
  print(i)
  neighbors = set()
  for node in nodes_to_find_neighbors:
    # Use the neighbors() function to get 1-hop neighbors and add them to the set
    neighbors.update(networkx_graph.neighbors(node))
    neighbors.update(networkx_graph.predecessors(node))

    if i == 0:
      G.add_node(node)

      for neighbor in networkx_graph.neighbors(node):
        # if neighbor not in nodes_to_find_neighbors:
        G.add_edge(node, neighbor)

      # for neighbor in networkx_graph.predecessors(node):
      #   # if neighbor not in nodes_to_find_neighbors:
      #   G.add_edge(neighbor, node)


    else:
      for neighbor in networkx_graph.neighbors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(node, neighbor)

      for neighbor in networkx_graph.predecessors(node):
        if neighbor not in up_to_k_hop_neighbors_list:
          G.add_edge(neighbor, node)




  # ensure the disjoint property
  if i > 0:
    for ele in up_to_k_hop_neighbors:
      neighbors_list = list(set(list(neighbors)).difference(set(ele)))
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))

  if i == 0:
    neighbors_list = list(set(neighbors).difference(set(tested_elements)))
    up_to_k_hop_neighbors_list += tested_elements

  nodes_to_find_neighbors = neighbors_list
  up_to_k_hop_neighbors.append(neighbors_list)
  up_to_k_hop_neighbors_list += neighbors_list

0


In [ ]:
len(tested_elements)

35843

In [ ]:
import pickle
# Assuming you have a NetworkX graph called 'graph'
with open("gdrive/My Drive/GNN_Sparsification/15%_Induced_SubG_Yelp.pkl", "wb") as f:
    pickle.dump(G, f)

In [ ]:
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

import time

# Measure the execution time
start_time = time.time()

partition = compute_maximum_bisimulation(G)
len(partition)

end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")

Number of nodes: 273468
Number of edges: 736892
Execution time: 23.249207735061646 seconds


In [ ]:
def find_tuple_index_with_integer(list_of_tuples, integer):
    return next((index for index, tup in enumerate(list_of_tuples) if integer in tup), -1)

In [ ]:
# Function to split long tuples
def split_long_tuples(lst, max_length):
    new_list = []
    for tup in lst:
        if len(tup) > max_length:
            # Split the tuple into smaller tuples
            split_tuples = [tup[i:i+max_length] for i in range(0, len(tup), max_length)]
            new_list.extend(split_tuples)
        else:
            new_list.append(tup)
    return new_list

# Split tuples with length over 10K into tuples with length 10K or under
max_length = 300
new_partition = split_long_tuples(partition, max_length)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from scipy.spatial.distance import euclidean
from scipy.cluster.hierarchy import linkage, fcluster
import numpy as np

def exclude_tested_elements_from_MB(partition, tested_elements):
    tuple_dict = {}
    indices = []

    tuple_list = partition

    # Sort the list by the number of elements in each tuple
    sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

    list_of_lists = [list(t) for t in sorted_list]

    # Convert tuples to lists and create a dictionary with elements as keys and their corresponding tuple index as values
    for idx, tpl in enumerate(list_of_lists):
        tpl_list = list(tpl)
        for ele in tpl_list:
            tuple_dict[ele] = idx
        list_of_lists[idx] = tpl_list

    # Update the dictionary and indices based on tested elements
    for ele in tested_elements:
        if ele in tuple_dict:
            idx = tuple_dict[ele]
            if len(list_of_lists[idx]) > 1:
                list_of_lists[idx].remove(ele)
                list_of_lists.append([ele])
                tuple_dict[ele] = len(list_of_lists) - 1
                indices.append(len(list_of_lists) - 1)
            else:
                indices.append(idx)

    return list_of_lists, indices

def alpha_Sparsification(list_of_lists, data, alpha):
  list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
  i = 0
  while len(list_of_lists[i])>1:

      Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
      threshold = alpha * np.max(Z[:, 2])
      clusters = fcluster(Z, threshold, criterion='distance')

      # Get cluster labels for each data point
      labels = clusters


      # Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='euclidean')
      # threshold = alpha * np.max(Z[:, 2])
      # clusters = fcluster(Z, threshold, criterion='distance')

      # # Get cluster labels for each data point
      # labels = clusters




      # Sample tuple to split
      data_tuple = list_of_tuples[i]

      # Create a dictionary to store split data
      split_data = {}

      # Iterate through the labels and tuple elements
      for label, value in zip(labels, data_tuple):
        if label not in split_data:
          split_data[label] = []
        split_data[label].append(value)

      # Convert the dictionary to a list of tuples (if needed)
      split_tuples = [tuple(values) for key, values in split_data.items()]
      list_of_tuples.pop(i)
      j = 0
      for values in split_tuples:
        if j == 0:
          list_of_tuples.insert(0, values)
        else:
          list_of_tuples.insert(len(list_of_tuples), values)
        j+=1

      i+=1

  result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

  return list_of_tuples, result_dict

# def alpha_Sparsification(list_of_lists, data, alpha):
#     list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]
#     i = 0
#     while i<len(list_of_lists):
#         if len(list_of_lists[i])>1:
#             Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
#             threshold = alpha * np.max(Z[:, 2])
#             clusters = fcluster(Z, threshold, criterion='distance')
#             # Get cluster labels for each data point
#             labels = clusters
#             # Sample tuple to split
#             data_tuple = list_of_tuples[i]
#             # Create a dictionary to store split data
#             split_data = {}
#           # Iterate through the labels and tuple elements
#             for label, value in zip(labels, data_tuple):
#                 if label not in split_data:
#                     split_data[label] = []
#                 split_data[label].append(value)

#           # Convert the dictionary to a list of tuples (if needed)
#             split_tuples = [tuple(values) for key, values in split_data.items()]
#             list_of_tuples.pop(i)
#             j = 0
#             for values in split_tuples:
#                 if j == 0:
#                     list_of_tuples.insert(0, values)
#                 else:
#                     list_of_tuples.insert(len(list_of_tuples), values)
#                 j+=1

#         i+=1

#     result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
#     return list_of_tuples, result_dict

In [ ]:
def Sparisified_G_NX(list_of_tuples, result_dict, data):
  BCG = nx.DiGraph()

  for i in range(len(list_of_tuples)):
    BCG.add_node(i)

  for edge in G.edges():
      source, target = edge
      # if cosine_similarity(data.x[source].numpy().reshape(1, -1), data.x[target].numpy().reshape(1, -1)) > 0.5:
      BCG.add_edge(result_dict[source], result_dict[target])
      # BCG.add_edge(result_dict[target], result_dict[source])

  num_nodes = BCG.number_of_nodes()
  num_edges = BCG.number_of_edges()



  print("Number of nodes:", num_nodes)
  print("Number of edges:", num_edges)


  return BCG

In [ ]:
def Sparisified_G_Alpha(BCG, data, list_of_tuples):
  num_nodes = BCG.number_of_nodes()
  new_x = torch.zeros(num_nodes,data.x.shape[1])
  new_y = torch.zeros(num_nodes, data.y.shape[1])
  for i in range(len(list_of_tuples)):
    new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    new_y[i,:] = torch.mean(data.y[list(list_of_tuples[i]),:], axis = 0)
    # tensor = torch.mean(data.y[list(list_of_tuples[i]),:], axis = 0)
    # original_tensor = np.array([[0.2, 0.6, 0.8],[0.4, 0.3, 0.7]])
    # new_tensor = (original_tensor > 0.5).astype(int)
    # mask = tensor > 0.5
    # new_y[i,:] = mask.astype(float)
    # EC = list_of_tuples[i]
    # L = []
    # for ele in EC:
    #   L.append(int(data.y[ele].numpy()))

    # keys = list(Counter(L).keys())
    # frequencies = list(Counter(L).values())

    # # Randomly select a key based on frequencies
    # random_key = random.choices(keys, frequencies)[0]
    # new_y[i] = int(random_key)

  edges = list(BCG.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

  data_new1 = copy.copy(data)
  data_new1.x = new_x
  data_new1.y = new_y.long()
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)



  # for u, v in BCG.edges():
  #   similarity = cosine_similarity(new_x[u].numpy().reshape(1, -1), new_x[v].numpy().reshape(1, -1))
  #   BCG[u][v]['weight'] = similarity[0][0]

  # edge_weights = [BCG[u][v]['weight'] for u, v in BCG.edges()]
  # # Convert edge weights to a PyTorch tensor
  # edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)

  return data_new1

In [ ]:
def Sparisified_G_Alpha_0(G, data):

  new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
  new_y = torch.zeros(len(list(G.nodes)),data.y.shape[1])
  list_itr = list(G.nodes)
  for i in range(len(list(G.nodes))):
    new_x[i,:] = data.x[list_itr[i],:]
    # new_y[i] = int(data.y[list_itr[i]])
    new_y[i,:] = data.y[list_itr[i],:]


  edges = list(G.edges())

  # Create a dictionary to map node labels to integer indices
  node_mapping = {node: index for index, node in enumerate(G.nodes())}

  # Convert the edges to indices using the node_mapping
  new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
  new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


  data_new1 = copy.copy(data)
  data_new1.x = new_x
  # data_new1.y = new_y.long()
  data_new1.y = new_y
  data_new1.edge_index = new_edge_index
  data_new1.num_nodes = len(new_y)

  # for u, v in G.edges():
  #   similarity = cosine_similarity(data.x[u].numpy().reshape(1, -1), data.x[v].numpy().reshape(1, -1))
  #   G[u][v]['weight'] = similarity[0][0]

  # edge_weights = [G[u][v]['weight'] for u, v in G.edges()]
  # # Convert edge weights to a PyTorch tensor
  # edge_weight_tensor = torch.tensor(edge_weights, dtype=torch.float32)

  return data_new1

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [1]:
data.x

NameError: name 'data' is not defined

In [ ]:
has_nan = data..isnan(tensor).any()

if has_nan:
    print("The tensor contains NaN values.")
else:
    print("The tensor does not contain NaN values.")

NameError: name 'data' is not defined

In [ ]:
alpha_list = [0, 0.1, 1]
import copy

for alpha in alpha_list:
  # print(alpha)
  if alpha == 0:
    start_time = time.time()
    data_new_1 = Sparisified_G_Alpha_0(G, data)
    end_time = time.time()
    execution_time_SPGS = end_time - start_time
    print(f"Execution time: {execution_time_SPGS} seconds")

    tested_elements_set = set(tested_elements)
    indices = [index for index, node in enumerate(G.nodes) if node in tested_elements_set]


  if alpha > 0 and alpha < 1:
    print(alpha)
    if alpha == alpha_list[1]:
      start_time_1 = time.time()
      list_of_lists, indices = exclude_tested_elements_from_MB(partition, tested_elements)


    list_of_tuples, result_dict = alpha_Sparsification(list_of_lists, data, alpha)
    # end_time_1 = time.time()
    # execution_time_1 = end_time_1 - start_time_1
    # print(f"Execution time of alpha_Sparsification: {execution_time_1} seconds")

    BCG = Sparisified_G_NX(list_of_tuples, result_dict, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, list_of_tuples)
    end_time_2 = time.time()
    execution_time_2 = end_time_2 - start_time_1
    print(f"Execution time of  (alpha, r) SPGC: {execution_time_2} seconds")

  if alpha == 1:
    l_o_t = [tuple(inner_list) for inner_list in list_of_lists]
    r_d = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}
    start_time_3 = time.time()
    BCG = Sparisified_G_NX(l_o_t, r_d, data)
    data_new_1 = Sparisified_G_Alpha(BCG, data, l_o_t)
    end_time_3 = time.time()
    execution_time_3 = end_time_3 - start_time_3
    print(f"Execution time of SPGC: {execution_time_3} seconds")


Execution time: 18.784494876861572 seconds
0.1


In [ ]:
tuple_list = partition
def find_tuple_index_with_integer(list_of_tuples, integer):
    return next((index for index, tup in enumerate(list_of_tuples) if integer in tup), -1)

# Sort the list by the number of elements in each tuple
sorted_list = sorted(tuple_list, key=lambda x: len(x), reverse=True)

indices = []
list_of_lists = [list(t) for t in sorted_list]

for ele in tested_elements:
  id = find_tuple_index_with_integer(sorted_list, ele)
  if len(list_of_lists[id]) > 1:
    list_of_lists[id].remove(ele)
    list_of_lists.append([ele])
    indices.append(len(list_of_lists)-1)
  else:
    indices.append(id)

list_of_tuples = [tuple(inner_list) for inner_list in list_of_lists]

In [ ]:
i = 0
while len(list_of_lists[i])>1:
    # print(i)

    Z = linkage(data.x[list_of_tuples[i],:], method='average', metric='cosine')
    threshold = 0.6 * np.max(Z[:, 2])
    clusters = fcluster(Z, threshold, criterion='distance')

    # Get cluster labels for each data point
    labels = clusters

    # Sample tuple to split
    data_tuple = list_of_tuples[i]

    # Create a dictionary to store split data
    split_data = {}

    # Iterate through the labels and tuple elements
    for label, value in zip(labels, data_tuple):
      if label not in split_data:
        split_data[label] = []
      split_data[label].append(value)

    # Convert the dictionary to a list of tuples (if needed)
    split_tuples = [tuple(values) for key, values in split_data.items()]
    list_of_tuples.pop(i)
    j = 0
    for values in split_tuples:
      if j == 0:
        list_of_tuples.insert(0, values)
      else:
        list_of_tuples.insert(len(list_of_tuples), values)
      j+=1

    i+=1

result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

# Alpha = 0.1


In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 74520
Number of edges: 185135


# Alpha = 0.2

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 54414
Number of edges: 177409


# Alpha = 0.3

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 43738
Number of edges: 161673


# 0.4

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 39537
Number of edges: 143608


# 0.5

In [ ]:
BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 37687
Number of edges: 134284


# 1

In [ ]:
result_dict = {element: index for index, tup in enumerate(list_of_tuples) for element in tup}

BCG = nx.DiGraph()
import random

for i in range(len(list_of_tuples)):
  BCG.add_node(i)

for edge in G.edges():
    source, target = edge
    BCG.add_edge(result_dict[source], result_dict[target])

# Get the number of nodes and edges
num_nodes = BCG.number_of_nodes()
num_edges = BCG.number_of_edges()

print("Number of nodes:", num_nodes)
print("Number of edges:", num_edges)

Number of nodes: 35114
Number of edges: 122843


In [ ]:
from collections import Counter
import random

new_x = torch.zeros(num_nodes,data.x.shape[1])
new_y = torch.zeros(num_nodes)
for i in range(len(list_of_tuples)):
  new_x[i,:] = torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
  EC = list_of_tuples[i]
  L = []
  for ele in EC:
    L.append(int(data.y[ele].numpy()))

  keys = list(Counter(L).keys())
  frequencies = list(Counter(L).values())

  # Randomly select a key based on frequencies
  random_key = random.choices(keys, frequencies)[0]
  new_y[i] = int(random_key)

edges = list(BCG.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
from collections import Counter
import random
import copy
from collections import Counter

new_x = torch.zeros(len(list(G.nodes)),data.x.shape[1])
new_y = torch.zeros(len(list(G.nodes)))
list_itr = list(G.nodes)
for i in range(len(list(G.nodes))):
  new_x[i,:] = data.x[list_itr[i],:]
  new_y[i] = int(data.y[list_itr[i]])

edges = list(G.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(G.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()


data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
# import torch

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super().__init__()

#     self.conv1 = GCNConv(dataset.num_node_features, 128,aggr = 'mean')
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GCNConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GCNConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GCN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# print("Graph Convolutional Network (GCN):")
# GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 40)
  (bc3): BatchNorm(40)
)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, GraphSAGE, BatchNorm
import torch

class GCN(torch.nn.Module):
  def __init__(self):
    super().__init__()

    self.conv1 = GCNConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GCNConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GCNConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)
  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("Graph Convolutional Network (GCN):")
GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(300, 128)
  (bc1): BatchNorm(128)
  (conv2): GCNConv(128, 128)
  (bc2): BatchNorm(128)
  (conv3): GCNConv(128, 100)
  (bc3): BatchNorm(100)
)

In [ ]:
# from torch_geometric.nn import GATConv, GraphSAGE, GCNConv
# import torch

# class GCN(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = GCNConv(dataset.num_node_features, 128, aggr = "mul")
#     self.conv2 = GCNConv(128, dataset.num_classes, aggr = "mul")
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GCN()
# # model = GCN()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("Graph Convolutional Network (GCN):")
# GCN()

Graph Convolutional Network (GCN):


GCN(
  (conv1): GCNConv(128, 128)
  (conv2): GCNConv(128, 40)
)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GraphSAGE(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = SAGEConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = SAGEConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = SAGEConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GraphSAGE()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
print("GraphSAGE:")
GraphSAGE()

GraphSAGE:


GraphSAGE(
  (conv1): SAGEConv(300, 128, aggr=mean)
  (bc1): BatchNorm(128)
  (conv2): SAGEConv(128, 128, aggr=mean)
  (bc2): BatchNorm(128)
  (conv3): SAGEConv(128, 100, aggr=mean)
  (bc3): BatchNorm(100)
)

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, SAGEConv
# import torch

# class GraphSAGE(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = SAGEConv(dataset.num_node_features, 128)
#     # self.conv2 = GCNConv(40, 20)
#     self.conv2 = SAGEConv(128, dataset.num_classes)
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matri
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)

#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GraphSAGE()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("GraphSAGE:")
# GraphSAGE()

In [ ]:
# from torch_geometric.nn import GATConv, GCNConv, SAGEConv
# import torch

# class GAT(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = GATConv(dataset.num_node_features, 128,aggr = 'mean')
#     self.bc1 = BatchNorm(128)
#     self.conv2 = GATConv(128, 128,aggr = 'mean')
#     self.bc2 = BatchNorm(128)
#     self.conv3 = GATConv(128, dataset.num_classes,aggr = 'mean')
#     self.bc3 = BatchNorm(dataset.num_classes)

#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc1(x)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc2(x)
#     x = self.conv3(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     x = self.bc3(x)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GAT()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("GAT:")
# GAT()

GAT:


GAT(
  (conv1): GATConv(128, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 40, heads=1)
  (bc3): BatchNorm(40)
)

In [ ]:
from torch_geometric.nn import GATConv, GCNConv, SAGEConv
import torch

class GAT(torch.nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = GATConv(dataset.num_node_features, 128)
    self.bc1 = BatchNorm(128)
    self.conv2 = GATConv(128, 128)
    self.bc2 = BatchNorm(128)
    self.conv3 = GATConv(128, dataset.num_classes)
    self.bc3 = BatchNorm(dataset.num_classes)

  def forward(self, x, edge_index):
    # x: Node feature matrix
    # edge_index: Graph connectivity matrix
    x = self.conv1(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc1(x)
    x = self.conv2(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc2(x)
    x = self.conv3(x, edge_index)
    x = torch.nn.functional.relu(x)
    x = self.bc3(x)
    return torch.nn.functional.log_softmax(x, dim=1)

model = GAT()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
print("GAT:")
GAT()

GAT:


GAT(
  (conv1): GATConv(300, 128, heads=1)
  (bc1): BatchNorm(128)
  (conv2): GATConv(128, 128, heads=1)
  (bc2): BatchNorm(128)
  (conv3): GATConv(128, 100, heads=1)
  (bc3): BatchNorm(100)
)

In [ ]:
# from torch_geometric.nn import GraphSAGE, GATConv
# import torch

# class GAT(torch.nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.conv1 = GATConv(dataset.num_node_features, 80, aggr = 'mean')
#     # self.conv2 = GCNConv(40, 20)
#     self.conv2 = GATConv(80, dataset.num_classes, aggr = 'mean')
#   def forward(self, x, edge_index):
#     # x: Node feature matrix
#     # edge_index: Graph connectivity matrix
#     x = self.conv1(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     # x = torch.nn.functional.dropout(x, training=self.training)
#     x = self.conv2(x, edge_index)
#     x = torch.nn.functional.relu(x)
#     # x = self.conv3(x, edge_index)
#     # x = torch.nn.functional.relu(x)
#     # # x = self.conv4(x, edge_index)
#     return torch.nn.functional.log_softmax(x, dim=1)

# model = GAT()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
# print("Graph Attention Networks(GAT):")
# GAT()

Graph Attention Networks(GAT):


GAT(
  (conv1): GATConv(128, 80, heads=1)
  (conv2): GATConv(80, 40, heads=1)
)

# Pre-trained GCN

In [ ]:
# train the model
import time
from sklearn.metrics import f1_score


# Start recording the current time
start_time = time.time()
criterion = torch.nn.BCEWithLogitsLoss()

model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  predictions = out[~test_mask]
  ground_truth = data.y[~test_mask]
  loss = criterion(predictions, ground_truth)
  losses.append(loss.item())
  # accuracies.append(acc)
  loss.backward()
  optimizer.step()
  # print(loss.item())
  if (epoch+1) % 10 == 0:
    print(epoch)
    print(loss.item())
  #   predicted_labels = out.detach().numpy() > 0.5
  #   true_labels = data.y.numpy()
  #   f1 = f1_score(true_labels, predicted_labels, average='micro')
  #   print("F1 Score:", f1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")


9
0.43699732422828674
19
0.4283868670463562
29
0.4214119613170624
39
0.4162753224372864
49
0.41207191348075867
59
0.40839844942092896
69
0.4051590859889984
79
0.40248486399650574
89
0.40032196044921875
99
0.39860594272613525


KeyboardInterrupt: ignored

In [ ]:
predicted_labels = out.detach().numpy() >= 0
true_labels = data.y.numpy()
f1 = f1_score(true_labels, predicted_labels.astype(float),  average='micro')
print("F1 Score:", f1)


F1 Score: 0.0


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score

# Define the neural network architecture
class MultiTargetBinaryClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(MultiTargetBinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No sigmoid activation here, BCEWithLogitsLoss applies it internally
        return x

# Example data (features and targets)
input_size = 10  # Number of input features
hidden_size1 = 32
hidden_size2 = 16
output_size = 5  # Number of binary targets

# Dummy data (random tensors)
num_samples = 1000
features = torch.randn(num_samples, input_size)
targets = torch.randint(2, size=(num_samples, output_size), dtype=torch.float32)  # Binary targets (0 or 1)

# Create an instance of the model
model = MultiTargetBinaryClassifier(input_size, hidden_size1, hidden_size2, output_size)

# Define the loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary Cross-Entropy Loss with Logits
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 1000
for epoch in range(num_epochs):
    # Forward pass
    predictions = model(features)
    loss = criterion(predictions, targets)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print the loss every few epochs
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Apply sigmoid activation and thresholding
sigmoid = nn.Sigmoid()
predicted_probs = sigmoid(predictions)
predicted_labels = torch.where(predicted_probs > 0.5, torch.tensor(1.0), torch.tensor(0.0))

# Convert predictions to numpy arrays for F1 score calculation
predicted_labels = predicted_labels.cpu().numpy()
true_labels = targets.cpu().numpy()

# Calculate F1 score for the predictions
f1 = f1_score(true_labels, predicted_labels, average='micro')

print("F1 Score:", f1)


Epoch [100/1000], Loss: 0.6791
Epoch [200/1000], Loss: 0.6545
Epoch [300/1000], Loss: 0.6255
Epoch [400/1000], Loss: 0.6011
Epoch [500/1000], Loss: 0.5801
Epoch [600/1000], Loss: 0.5622
Epoch [700/1000], Loss: 0.5482
Epoch [800/1000], Loss: 0.5399
Epoch [900/1000], Loss: 0.5335
Epoch [1000/1000], Loss: 0.5280
F1 Score: 0.7355454003576396


In [ ]:
predictions

tensor([[-1.6417, -1.1763,  0.3937, -0.8896, -3.0917],
        [-0.6852,  0.4234,  0.1621, -1.7398,  1.1862],
        [-0.0929,  0.0703, -0.7624, -0.7826,  0.7290],
        ...,
        [-0.0372, -0.4700, -0.8579,  0.0625, -0.3404],
        [ 0.2310, -0.6194,  2.1844,  0.4971,  1.3415],
        [ 0.9822,  0.5719,  0.3667,  0.1250,  0.7537]],
       grad_fn=<AddmmBackward0>)

In [ ]:
sigmoid = nn.Sigmoid()
predicted_probs = sigmoid(out)
predicted_labels = torch.where(predicted_probs > 0.5, torch.tensor(1.0), torch.tensor(0.0))

# Convert predictions to numpy arrays for F1 score calculation
predicted_labels = predicted_labels.cpu().numpy()
true_labels = data.y.numpy()

# Calculate F1 score for the predictions
f1 = f1_score(true_labels, predicted_labels, average='micro')

print("F1 Score:", f1)

F1 Score: 0.0


In [ ]:
predicted_probs

tensor([[0.0176, 0.0218, 0.0270,  ..., 0.0043, 0.0037, 0.0073],
        [0.0117, 0.0074, 0.0050,  ..., 0.0048, 0.0060, 0.0026],
        [0.0122, 0.0582, 0.0336,  ..., 0.0046, 0.0040, 0.0068],
        ...,
        [0.0273, 0.0810, 0.0197,  ..., 0.0029, 0.0030, 0.0080],
        [0.0211, 0.0310, 0.0323,  ..., 0.0041, 0.0044, 0.0078],
        [0.0279, 0.0649, 0.0351,  ..., 0.0036, 0.0038, 0.0111]],
       grad_fn=<SigmoidBackward0>)

In [ ]:
# # train the model
# def compute_accuracy(pred_y, y):
#     return (pred_y == y).sum()

# import time

# # Start recording the current time
# start_time = time.time()


# model.train()
# losses = []
# accuracies = []
# for epoch in range(300):
#   optimizer.zero_grad()
#   out = model(data.x, data.edge_index)
#   loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
#   correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
#   acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
#   losses.append(loss.item())
#   accuracies.append(acc)
#   loss.backward()
#   optimizer.step()
#   if (epoch+1) % 10 == 0:
#     print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# # End recording the current time
# end_time = time.time()

# # Calculate the elapsed time
# elapsed_time = end_time - start_time

# print("Training time:", elapsed_time, "seconds")

# torch.save(model.state_dict(), model_save_path_GCN)

# Pre-trained GAT

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
  correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
  acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
  losses.append(loss.item())
  accuracies.append(acc)
  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

torch.save(model.state_dict(), model_save_path_GAT)

Epoch: 10, Loss: 2.2442, Training Acc: 0.5062
Epoch: 20, Loss: 1.9214, Training Acc: 0.5587
Epoch: 30, Loss: 1.7382, Training Acc: 0.5803
Epoch: 40, Loss: 1.6119, Training Acc: 0.5976
Epoch: 50, Loss: 1.5174, Training Acc: 0.6097
Epoch: 60, Loss: 1.4413, Training Acc: 0.6197
Epoch: 70, Loss: 1.3772, Training Acc: 0.6287
Epoch: 80, Loss: 1.3227, Training Acc: 0.6387
Epoch: 90, Loss: 1.2786, Training Acc: 0.6463
Epoch: 100, Loss: 1.2411, Training Acc: 0.6519
Epoch: 110, Loss: 1.2027, Training Acc: 0.6608
Epoch: 120, Loss: 1.1747, Training Acc: 0.6659
Epoch: 130, Loss: 1.1445, Training Acc: 0.6719
Epoch: 140, Loss: 1.1292, Training Acc: 0.6748
Epoch: 150, Loss: 1.1212, Training Acc: 0.6753
Epoch: 160, Loss: 1.0792, Training Acc: 0.6865
Epoch: 170, Loss: 1.0787, Training Acc: 0.6852
Epoch: 180, Loss: 1.0530, Training Acc: 0.6928
Epoch: 190, Loss: 1.0563, Training Acc: 0.6918
Epoch: 200, Loss: 1.0218, Training Acc: 0.7015
Epoch: 210, Loss: 1.0229, Training Acc: 0.6993
Epoch: 220, Loss: 1.00

# Pre-trained GraphSage

In [ ]:
# train the model
def compute_accuracy(pred_y, y):
    return (pred_y == y).sum()

import time

# Start recording the current time
start_time = time.time()


model.train()
losses = []
accuracies = []
for epoch in range(300):
  optimizer.zero_grad()
  out = model(data.x, data.edge_index)
  loss = torch.nn.functional.nll_loss(out[~test_mask], data.y[~test_mask])
  correct = compute_accuracy(out[~test_mask].argmax(dim=1), data.y[~test_mask])
  acc = int(correct) / (len(test_mask)-int(test_mask.int().sum()))
  losses.append(loss.item())
  accuracies.append(acc)
  loss.backward()
  optimizer.step()
  if (epoch+1) % 10 == 0:
    print('Epoch: {}, Loss: {:.4f}, Training Acc: {:.4f}'.format(epoch+1, loss.item(), acc))

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Training time:", elapsed_time, "seconds")

torch.save(model.state_dict(), model_save_path_GraphSage)

Epoch: 10, Loss: 2.0894, Training Acc: 0.5410
Epoch: 20, Loss: 1.8166, Training Acc: 0.5765
Epoch: 30, Loss: 1.6427, Training Acc: 0.5996
Epoch: 40, Loss: 1.5192, Training Acc: 0.6141
Epoch: 50, Loss: 1.4252, Training Acc: 0.6269
Epoch: 60, Loss: 1.3512, Training Acc: 0.6374
Epoch: 70, Loss: 1.2898, Training Acc: 0.6472
Epoch: 80, Loss: 1.2404, Training Acc: 0.6549
Epoch: 90, Loss: 1.1926, Training Acc: 0.6643
Epoch: 100, Loss: 1.1631, Training Acc: 0.6679
Epoch: 110, Loss: 1.1217, Training Acc: 0.6772
Epoch: 120, Loss: 1.0903, Training Acc: 0.6844
Epoch: 130, Loss: 1.0596, Training Acc: 0.6919
Epoch: 140, Loss: 1.0356, Training Acc: 0.6976
Epoch: 150, Loss: 1.0120, Training Acc: 0.7031
Epoch: 160, Loss: 0.9963, Training Acc: 0.7061
Epoch: 170, Loss: 0.9632, Training Acc: 0.7151
Epoch: 180, Loss: 0.9482, Training Acc: 0.7189
Epoch: 190, Loss: 0.9407, Training Acc: 0.7198
Epoch: 200, Loss: 0.9155, Training Acc: 0.7272
Epoch: 210, Loss: 0.9131, Training Acc: 0.7273
Epoch: 220, Loss: 0.88

# GCN

In [ ]:
# evaluate the model on test set
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))


import time
start_time = time.time()
model.eval()
pred = model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 1.3753893375396729 seconds
Accuracy: 0.6263


# alpha = 0

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.47821044921875 seconds
Accuracy: 0.5570


# alpha = 0.1

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.4155430793762207 seconds
Accuracy: 0.5499


# alpha = 0.2

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2500901222229004 seconds
Accuracy: 0.5401


# 0.3

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.237440824508667 seconds
Accuracy: 0.5273


# 0.4

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.17484664916992188 seconds
Accuracy: 0.5222


# 0.5

In [ ]:
best_model = GCN()
best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1378035545349121 seconds
Accuracy: 0.5202


# alpha = 1

In [ ]:
from collections import Counter
import random

new_x = torch.zeros(num_nodes,data.x.shape[1])
new_y = torch.zeros(num_nodes)
for i in range(len(list_of_tuples)):
    sum_node_degree = 0
    for ele in list_of_tuples[i]:
        sum_node_degree += G.out_degree(ele)
    new_node_degree = BCG.out_degree(i)
    if new_node_degree != 0:
        scalar = sum_node_degree/new_node_degree

    else:
        scalar = 1


    new_x[i,:] = scalar * torch.mean(data.x[list(list_of_tuples[i]),:], axis = 0)
    EC = list_of_tuples[i]
    L = []
    for ele in EC:
        L.append(int(data.y[ele].numpy()))

    keys = list(Counter(L).keys())
    frequencies = list(Counter(L).values())

    # Randomly select a key based on frequencies
    random_key = random.choices(keys, frequencies)[0]
    new_y[i] = int(random_key)

edges = list(BCG.edges())

# Create a dictionary to map node labels to integer indices
node_mapping = {node: index for index, node in enumerate(BCG.nodes())}

# Convert the edges to indices using the node_mapping
new_edge_index = [[node_mapping[source], node_mapping[target]] for source, target in edges]
new_edge_index = torch.tensor(new_edge_index, dtype=torch.long).t().contiguous()

data_new1 = copy.copy(data)
data_new1.x = new_x
data_new1.y = new_y.long()
data_new1.edge_index = new_edge_index
data_new1.num_nodes = len(new_y)

In [ ]:
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.09882307052612305 seconds
Accuracy: 0.5013


In [ ]:
# best_model = GCN()
# best_model.load_state_dict(torch.load(model_save_path_GCN))

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1014864444732666 seconds
Accuracy: 0.5004


# GAT

In [ ]:
# evaluate the model on test set
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

import time
start_time = time.time()
best_model.eval()
pred = best_model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 1.5979797840118408 seconds
Accuracy: 0.6168


# Alpha = 0

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.579277753829956 seconds
Accuracy: 0.5684


# Alpha = 0.1

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.37201881408691406 seconds
Accuracy: 0.5652


# Alpha = 0.2

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2746572494506836 seconds
Accuracy: 0.5511


# 0.3

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.2083132266998291 seconds
Accuracy: 0.5353


# 0.4

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.18581390380859375 seconds
Accuracy: 0.5284


# 1

In [ ]:
best_model = GAT()
best_model.load_state_dict(torch.load(model_save_path_GAT))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.20810818672180176 seconds
Accuracy: 0.5190


In [ ]:
(0.5725-0.5224)/0.5725

0.08751091703056774

In [ ]:
model_save_path_GraphSage

'gdrive/My Drive/GNN_Sparsification/Pre_Trained_5%_ogbn_arxiv_GraphSage.pt'

# GraphSage

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

# evaluate the model on test set
import time
start_time = time.time()
best_model.eval()
pred = best_model(data.x, data.edge_index).argmax(dim=1)
# correct = compute_accuracy(pred, test_data.y)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[test_mask], data.y[test_mask])
# acc = int(correct) / len(test_data.y)
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 1.4965898990631104 seconds
Accuracy: 0.6111


# Alpha = 0

In [ ]:
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

In [ ]:
# evaluate the model on test set
start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.22997593879699707 seconds
Accuracy: 0.5800


# Alpha = 0.1

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.1699836254119873 seconds
Accuracy: 0.5762


# Alpha = 0.2


In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.10684919357299805 seconds
Accuracy: 0.5712


# 0.3

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.06638884544372559 seconds
Accuracy: 0.5622


# 0.4

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.08106851577758789 seconds
Accuracy: 0.5577


# 1

In [ ]:
best_model = GraphSAGE()
best_model.load_state_dict(torch.load(model_save_path_GraphSage))

start_time = time.time()
best_model.eval()
pred = best_model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.10892224311828613 seconds
Accuracy: 0.5505


In [ ]:
# evaluate the model on test set
list2 = tested_elements
list1 = list(G.nodes)

indices = [list1.index(x) for x in list2]

start_time = time.time()
model.eval()
pred = model(data_new1.x, data_new1.edge_index).argmax(dim=1)

# End recording the current time
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

print("Inference time:", elapsed_time, "seconds")
correct = compute_accuracy(pred[indices], data_new1.y[indices])
acc = int(correct) / len(tested_elements)
print(f'Accuracy: {acc:.4f}')

Inference time: 0.164994478225708 seconds
Accuracy: 0.5691
